In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import time
import json
print("Import successful")

Import successful


In [2]:
URI = os.environ["NEO4J_URI"]
USER=os.environ["NEO4J_USER_NAME"]
PASSWORD=os.environ["NEO4J_PASSWD"]
AUTH = (os.environ["NEO4J_USER_NAME"], os.environ["NEO4J_PASSWD"])

In [3]:
#Neo4J connect and Query Boilerplate

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
            
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            #response = (session.run(query, parameters))
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        
        #return pd.DataFrame([r.values() for r in response], columns=response.keys())
        return response
        
    def multi_query(self, multi_line_query, parameters=None, db=None):
        for li in multi_line_query.splitlines():
                print(li)
                result=self.query(li, parameters=None, db=None)
                print(result)

In [4]:
#Make a default connection and it should return `[<Record count(n)=0>]`
conn = Neo4jConnection(uri=URI, 
                       user=USER,
                       pwd=PASSWORD)

#if db is empty, then seed with init values 
res=conn.query('MATCH (n) RETURN count(n)')
print(res)

[<Record count(n)=13776>]


In [ ]:
# NOTE: should be executed only once per database; Alternatively, you can also copy paste this query into neo4j browser
constraint = [
"CREATE CONSTRAINT ing_id FOR (ing:Ingredient) REQUIRE ing.itemID IS UNIQUE;",
"CREATE CONSTRAINT diabetes_cat_id FOR (diab_cat:DiabetesCategory) REQUIRE diab_cat.itemID IS UNIQUE;",
"CREATE CONSTRAINT diabetes_label_id FOR (diab_des:DiabetesDecision) REQUIRE diab_des.itemID IS UNIQUE;",
"CREATE CONSTRAINT contains_id FOR (contains:Contains) REQUIRE contains.itemID IS UNIQUE;",
"CALL db.awaitIndexes();"]

for cquery in constraint:
    try:
        res = conn.query(cquery)
        print(f"Executed successfully: {cquery}")
    except Exception as e:
        print(f"Error executing query: {cquery}\n{e}")

In [ ]:
# queries to load the data
load_ingredients = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/ingredients.csv') yield map as row", 
"MERGE (ing:Ingredient {itemID: row.id}) 
ON CREATE SET ing.name = row.name, ing.USFDACategory = row.USFDA_Category, ing.nutrition = row.nutrition, ing.url = row.src_url", 
{batchSize:1000, iterateList:true, parallel:true})"""

load_diabetes_cat = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/diabetes_category.csv') yield map as row", 
"MERGE (diab_cat:DiabetesCategory {itemID: row.id}) 
ON CREATE SET diab_cat.name = row.label, diab_cat.knowledgeSource = row.disease_knowledge_name, diab_cat.url=row.disease_knowledge_url, diab_cat.description=row.description, diab_cat.sampleItems=row.example_items", 
{batchSize:1000, iterateList:true, parallel:true})"""

load_diab_label = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/diabetes_decision.csv') yield map as row", 
"MERGE (diab_des:DiabetesDecision {itemID: row.id}) 
ON CREATE SET diab_des.name = row.label", 
{batchSize:1000, iterateList:true, parallel:true})"""

load_contains = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/contains.csv') yield map as row", 
"MERGE (contains:Contains {itemID: row.id}) 
ON CREATE SET contains.name = row.name", 
{batchSize:1000, iterateList:true, parallel:true})"""

In [6]:
result = conn.query(load_ingredients)
print("Ingredient nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_diabetes_cat)
print("DiabetesCategory nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_diab_label)
print("DiabetesLabel nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_contains)
print("Contains nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Ingredient nodes loaded. Time taken:2 seconds. Committed Operations: 13538. Failed Operations:0
DiabetesCategory nodes loaded. Time taken:0 seconds. Committed Operations: 22. Failed Operations:0
DiabetesLabel nodes loaded. Time taken:0 seconds. Committed Operations: 3. Failed Operations:0
Contains nodes loaded. Time taken:0 seconds. Committed Operations: 26. Failed Operations:0


In [ ]:
load_ing_diab_cat="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/rel_ing_diab_cat.csv') yield map as row", 
"MATCH (ing:Ingredient {itemID: row.id1})
MATCH (diab_cat:DiabetesCategory {itemID: row.id2})
MERGE (ing)-[rel:belongsTo]->(diab_cat)
SET rel.tag = row.tag, 
    rel.allowed = row.allowed, 
    rel.present = row.present,
    rel.explanation = row.explanation", 
{batchSize:10000, iterateList:true, parallel:true})
"""

load_diab_cat_diab_des="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/rel_diab_cat_diab_des.csv') yield map as row", 
"MATCH (diab_cat:DiabetesCategory {itemID: row.id1})
MATCH (diab_des:DiabetesDecision {itemID: row.id2})
MERGE (diab_cat)-[:classifiedAs]->(diab_des)", 
{batchSize:10000, iterateList:true, parallel:true})
"""

load_ing_contain="""
CALL apoc.periodic.iterate("CALL apoc.load.csv('/DiabetesReasoning/rel_ing_contain.csv') yield map as row", 
"MATCH (ing:Ingredient {itemID: row.id1})
MATCH (contains:Contains {itemID: row.id2})
MERGE (ing)-[:has]->(contains)", 
{batchSize:10000, iterateList:true, parallel:true})
"""

In [25]:
result = conn.query(load_ing_diab_cat)
print("Relationship Ingredient-USFDACategory loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_diab_cat_diab_des)
print("Relationship DiabetesCategory-DiabetesDecision loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_ing_contain)
print("Relationship Ingredient-Contains loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Relationship Ingredient-USFDACategory loaded. Time taken:1 seconds. Committed Operations: 115278. Failed Operations:0
Relationship DiabetesCategory-DiabetesDecision loaded. Time taken:0 seconds. Committed Operations: 22. Failed Operations:0
Relationship Ingredient-Contains loaded. Time taken:0 seconds. Committed Operations: 122128. Failed Operations:0


In [ ]:
# add the reduced ingredient names - optional. But these are the names that are used to add carcinogens, smoke point and glycemic index
import sys
from tqdm import tqdm

id_query = """MATCH (n:Ingredient)
RETURN n.itemID AS itemID"""
res = conn.query(id_query)
ids =  [record["itemID"] for record in res]

reduced_ings = json.load(open('data/USFDA_Reduced_Ingredients_Chatgpt.json', 'r'))
done_ids = []
print("Adding reduced ingredient name as property")
for ing_id in tqdm(reduced_ings):
    reduced_name = reduced_ings[ing_id]['main_ing']
    set_query = """MATCH (n:Ingredient {itemID: $ing_id})
    SET n.reduced_name = $reduced_ing_name
    RETURN n"""
    parameters = {'ing_id':str(ing_id),'reduced_ing_name': reduced_name}
    res = conn.query(set_query, parameters)
    if len(res) != 0:
        done_ids.append(ing_id)
    
remaining = set(ids).difference(set(done_ids))
if len(remaining) == 0:
    print("Property is added successfully to all nodes")
else:
    print("Property is not added to the following nodes:")
    print(list(remaining))

Adding reduced ingredient name as property


100%|██████████| 13538/13538 [00:06<00:00, 2173.60it/s]

Property is added successfully to all nodes


In [ ]:
# Add measure units - optional
import sys
from tqdm import tqdm

id_query = """MATCH (n:Ingredient)
RETURN n.itemID AS itemID"""
res = conn.query(id_query)
ids =  [record["itemID"] for record in res]

measure_units = json.load(open('data/measure_units.json', 'r'))
done_ids = []
print("Adding reduced ingredient name as property")
for ing_id in tqdm(measure_units):
    curr_unit = json.dumps(measure_units[ing_id]) # serialize the list. To deserialize, do json.loads() when pulling data from neo4j
    set_query = """MATCH (n:Ingredient {itemID: $ing_id})
    SET n.measure_unit = $units
    RETURN n"""
    parameters = {'ing_id':str(ing_id),'units': curr_unit}
    res = conn.query(set_query, parameters)
    if len(res) != 0:
        done_ids.append(ing_id)
    
remaining = set(ids).difference(set(done_ids))
if len(remaining) == 0:
    print("Property is added successfully to all nodes")
else:
    print("Property is not added to the following nodes")
    print(remaining)

Adding reduced ingredient name as property


100%|██████████| 13539/13539 [00:18<00:00, 746.24it/s]

Property is added successfully to all nodes
